# Part Segmentation (ShapeNet)

## Jupyter Notebook implementation

First read the [README](README.md) file if you're new.

This is an example of using the code from Jupyter Notebook.

## Contents

- [Segmentation Classes](#segmentation-classes)
- [Training](#training)
- [Testing](#testing)
- [Inference](#inference)

## Imports

In [2]:
from pprint import pprint
from pathlib import Path
import json

import numpy as np
from tqdm import tqdm_notebook as tqdm
import torch
print(f'CUDA available?: {torch.cuda.is_available()}')

import train_partseg
import test_partseg
import infer_partseg

CUDA available?: True


## **Optional**: External Directory

If you want to put the log and data directories in a different location, you can use the following code.

In [3]:
root_log_dir = Path('..', 'log').resolve()
DATA_DIR = str(Path('..', 'data').resolve())
partseg_data_dir = str(Path(DATA_DIR, 'shapenetcore_partanno_segmentation_benchmark_v0_normal').resolve())

root_log_dir, DATA_DIR, partseg_data_dir

(PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'),
 '/home/gokitune/Documents/pointnet-cleanup/data',
 '/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal')

---

## Data

- ShapeNet dataset in text format
- Download `shapenetcore_partanno_segmentation_benchmark_v0_normal` from:
  - [Kaggle](https://www.kaggle.com/datasets/mitkir/shapenet?resource=download)


Default `--data_dir` is `'data/shapenetcore_partanno_segmentation_benchmark_v0_normal'`.

- `data/`
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/`
    - `02691156/`
      - `1a04e3eab45ca15dd86060f189eb133.txt`
      - `1a04e3eab45ca15dd86060f189eb133.npz`
      - ...
    - ...
    - `train_test_split/`
      - `shuffled_train_filelist.json`
      - `shuffled_val_filelist.json`
      - `shuffled_test_filelist.json`
    - `utils/`
      - `potential_field.py`
      - `som.py`
    - `synsetoffset2category.txt`

### Data structure

From `data/shapenetcore_partanno_segmentation_benchmark_v0_normal/synsetoffset2category.txt`, the folders correspond to the following categories:

- `Airplane`: 02691156
- `Bag`: 02773838
- `Cap`: 02954340
- `Car`: 02958343
- `Chair`: 03001627
- `Earphone`: 03261776
- `Guitar`: 03467517
- `Knife`: 03624134
- `Lamp`: 03636649
- `Laptop`: 03642806
- `Motorbike`: 03790512
- `Mug`: 03797390
- `Pistol`: 03948459
- `Rocket`: 04099429
- `Skateboard`: 04225987
- `Table`: 04379243

For each .txt file within the folder above, 

 - `[i, :]` is the i th point.
 - `[:, 0:3]` is xyz.
 - `[:, 3:6]` is normalized xyz.
 - `[:, 6]` is the segmentation label.

i.e., each row is a point, and the columns are `[x, y, z, nx, ny, nz, label]`.

`--normal` flag will use all x-y-z-nx-ny-nz + label as input. Otherwise, only x-y-z + label will be used.

#### **OUTPUT**

- TRAINING: `<log_root>/part_seg/<args.log_dir or TIME>/checkpoints/best_model.pth`


---

## Segmentation Classes

In [4]:
# shapenet part segmentation
seg_classes = {
    'Earphone'  : [16, 17, 18],
    'Motorbike' : [30, 31, 32, 33, 34, 35],
    'Rocket'    : [41, 42, 43],
    'Car'       : [8, 9, 10, 11],
    'Laptop'    : [28, 29],
    'Cap'       : [6, 7],
    'Skateboard': [44, 45, 46],
    'Mug'       : [36, 37],
    'Guitar'    : [19, 20, 21],
    'Bag'       : [4, 5],
    'Lamp'      : [24, 25, 26, 27],
    'Table'     : [47, 48, 49],
    'Airplane'  : [0, 1, 2, 3],
    'Pistol'    : [38, 39, 40],
    'Chair'     : [12, 13, 14, 15],
    'Knife'     : [22, 23]
}

seg_ids = [seg_id for seg_val_sublist in seg_classes.values() for seg_id in seg_val_sublist]
len(seg_classes), len(seg_ids)

(16, 50)

---

## Training

`train_partseg.py` is used to train the model.

Check all the arguments:

In [ ]:
!python3 train_partseg.py -h

The following is the same as running:

```bash
python3 train_partseg.py \
     --model pointnet2_part_seg_msg \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal\
```

In [ ]:
args = {
    'model'   : 'pointnet2_part_seg_msg',
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
    'notebook': True
}
partseg_train_args = train_partseg.CommandLineArgs(**args)
train_partseg.main(partseg_train_args, seg_classes)

---

## Training with DDP

In [ ]:
!python3 train_partseg_ddp.py -h

```bash
python3 train_partseg_ddp.py \
    --model pointnet2_part_seg_msg \
    --normal \
    --log_dir ../log/part_segpointnet2_part_seg_msg \
    --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
    --batch_size 32 \
    --world_size 1
```

---

## Testing

`test_partseg.py` is used to test the model.

Check all the arguments:

In [ ]:
!python3 test_partseg.py -h

The following is the same as running:

```bash
python3 test_partseg.py \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
```

In [ ]:
args = {
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
    'notebook': True
}
partseg_test_args = test_partseg.CommandLineArgs(**args)
test_metrics, shape_ious, total_correct_class, total_seen_class = test_partseg.main(partseg_test_args, seg_classes)

In [ ]:
test_metrics, shape_ious

In [ ]:
seg_correct = dict(zip(range(len(seg_ids)), total_correct_class))
seg_total = dict(zip(range(len(seg_ids)), total_seen_class))

seg_acc = {}
for id, correct_n in seg_correct.items():
    total_n = seg_total[id]
    if total_n == 0:
        seg_acc[id] = 0
    else:
        seg_acc[id] = correct_n / total_n
# print(seg_acc)

seg_class_acc = {}
for cat in seg_classes:
    seg_class_acc[cat] = {}
    for id in seg_classes[cat]:
        seg_class_acc[cat][id] = seg_acc[id]

pprint(seg_class_acc)

---

## Inference

`infer_partseg.py` is used for inference.

NOTE: This is not included in the original codebase so there are slight differences in configuration.

In [ ]:
!python3 infer_partseg.py -h

```bash
python3 infer_partseg.py \
    --normal \
    --batch_size 48 #\
    # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal \
    # --log_dir pointnet2_part_seg_msg \
```

In [14]:
# See infer_partseg.PointnetInference docstring
# fmt: off
config = {
    "gpu"          : 0,
    "batch_size"   : 48,
    "num_point"    : 2048,
    "normal"       : True,
    "num_votes"    : 3,
    "model_name"   : "pointnet2_part_seg_msg",
    "log_dir"      : root_log_dir / "part_seg" / "pointnet2_part_seg_msg",
    "num_parts"    : len(seg_ids),
    "num_classes"  : len(seg_classes),
}
config["out_path"] = config["log_dir"] / "inference_results"
config["pt_path"]  = config["log_dir"] / "checkpoints" / "best_model.pth"
# fmt: on

data_dir = Path(partseg_data_dir)
with open(data_dir / "train_test_split" / "shuffled_test_file_list.json", "r") as f:
    test_ids = list(set([str(d) for d in json.load(f)]))
test_ids = [
    data_dir / test_id.split("/")[-2] / f"{test_id.split('/')[-1]}.txt"
    for test_id in test_ids
]
test_ids

[PosixPath('/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/04379243/d187561a6b0cbd0acaed5ce7390f30b7.txt'),
 PosixPath('/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/03001627/6caccdad9f8d4f0a7f1cdfc0a8f38f2e.txt'),
 PosixPath('/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/04379243/2f98d5e721e84debaa8081a7009091db.txt'),
 PosixPath('/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/03001627/e6c11fed9469141ace8fba09dd640742.txt'),
 PosixPath('/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/02691156/b3a59a941500e76535592b447835a16e.txt'),
 PosixPath('/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/03636649/eb311e6232cb7011bb5bd941c6665c21.txt'),
 PosixPath('/home/goki

### With performance metrics (for validation purposes)

In [ ]:
test_metrics, shape_ious, total_correct_class, total_seen_class = infer_partseg.main(
    conf=config, data_paths=test_ids, segmentation_classes=seg_classes, notebook=True
)

In [ ]:
test_metrics, shape_ious

In [ ]:
seg_correct = dict(zip(range(len(seg_ids)), total_correct_class))
seg_total = dict(zip(range(len(seg_ids)), total_seen_class))

seg_acc = {}
for id, correct_n in seg_correct.items():
    total_n = seg_total[id]
    if total_n == 0:
        seg_acc[id] = 0
    else:
        seg_acc[id] = correct_n / total_n
# print(seg_acc)

seg_class_acc = {}
for cat in seg_classes:
    seg_class_acc[cat] = {}
    for id in seg_classes[cat]:
        seg_class_acc[cat][id] = seg_acc[id]

pprint(seg_class_acc)

### Inference only

In [32]:
label_cat = infer_partseg.LabelToCategory(seg_classes)
points =  infer_partseg.txt_path_to_batch_tensor([test_ids[0]], normals=True)
pn_inf = infer_partseg.PointnetInference(config)
pred, logit = pn_inf.infer(points[:, :-1, :], label_cat.label_to_onehot(points[:, :-1, :]))

PARAMETER ...
    gpu: 0
    batch_size: 48
    num_point: 2048
    normal: True
    num_votes: 3
    model_name: pointnet2_part_seg_msg
    log_dir: /home/gokitune/Documents/pointnet-cleanup/log/part_seg/pointnet2_part_seg_msg
    num_parts: 50
    num_classes: 16
    out_path: /home/gokitune/Documents/pointnet-cleanup/log/part_seg/pointnet2_part_seg_msg/inference_results
    pt_path: /home/gokitune/Documents/pointnet-cleanup/log/part_seg/pointnet2_part_seg_msg/checkpoints/best_model.pth


In [35]:
pred.T == points[:, -1, :]

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]], shape=(2048, 1))